# Análise Exploratória

## Bibliotecas necessarias + opções de visualização do Pandas

In [150]:
import pandas as pd
import numpy as np
import json
from urllib.request import urlopen

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 100
pd.options.display.float_format = '{:,.2f}'.format

notrandomseed = 484


## Recebendo os dados e abrindo em um DataFrame

In [151]:
data_url = "https://github.com/sthemonica/alura-voz/blob/main/Dados/Telco-Customer-Churn.json?raw=true"
response = urlopen(data_url)
data_json = json.loads(response.read())

df = pd.json_normalize(data_json, max_level=2, sep='_')
df.head()

,customerID,Churn,customer_gender,customer_SeniorCitizen,customer_Partner,customer_Dependents,customer_tenure,phone_PhoneService,phone_MultipleLines,internet_InternetService,internet_OnlineSecurity,internet_OnlineBackup,internet_DeviceProtection,internet_TechSupport,internet_StreamingTV,internet_StreamingMovies,account_Contract,account_PaperlessBilling,account_PaymentMethod,account_Charges_Monthly,account_Charges_Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.4


## Analisando superficialmento informações sobre o DataFrame

#### Dicionário de dados

* `customerID`: número de identificação único de cada cliente
* `Churn`: se o cliente deixou ou não a empresa 
* `gender`: gênero (masculino e feminino) 
* `SeniorCitizen`: informação sobre um cliente ter ou não idade igual ou maior que 65 anos 
* `Partner`:  se o cliente possui ou não um parceiro ou parceira
* `Dependents`: se o cliente possui ou não dependentes
* `tenure`:  meses de contrato do cliente
* `PhoneService`: assinatura de serviço telefônico 
* `MultipleLines`: assisnatura de mais de uma linha de telefone 
* `InternetService`: assinatura de um provedor internet 
* `OnlineSecurity`: assinatura adicional de segurança online 
* `OnlineBackup`: assinatura adicional de backup online 
* `DeviceProtection`: assinatura adicional de proteção no dispositivo 
* `TechSupport`: assinatura adicional de suporte técnico, menos tempo de espera
* `StreamingTV`: assinatura de TV a cabo 
* `StreamingMovies`: assinatura de streaming de filmes 
* `Contract`: tipo de contrato
* `PaperlessBilling`: se o cliente prefere receber online a fatura
* `PaymentMethod`: forma de pagamento
* `Charges.Monthly`: total de todos os serviços do cliente por mês
* `Charges.Total`: total gasto pelo cliente

In [152]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   object 
 1   Churn                      7267 non-null   object 
 2   customer_gender            7267 non-null   object 
 3   customer_SeniorCitizen     7267 non-null   int64  
 4   customer_Partner           7267 non-null   object 
 5   customer_Dependents        7267 non-null   object 
 6   customer_tenure            7267 non-null   int64  
 7   phone_PhoneService         7267 non-null   object 
 8   phone_MultipleLines        7267 non-null   object 
 9   internet_InternetService   7267 non-null   object 
 10  internet_OnlineSecurity    7267 non-null   object 
 11  internet_OnlineBackup      7267 non-null   object 
 12  internet_DeviceProtection  7267 non-null   object 
 13  internet_TechSupport       7267 non-null   objec

In [153]:
df.describe(include='object')

,customerID,Churn,customer_gender,customer_Partner,customer_Dependents,phone_PhoneService,phone_MultipleLines,internet_InternetService,internet_OnlineSecurity,internet_OnlineBackup,internet_DeviceProtection,internet_TechSupport,internet_StreamingTV,internet_StreamingMovies,account_Contract,account_PaperlessBilling,account_PaymentMethod,account_Charges_Total
count,7267,7267,7267,7267,7267,7267,7267,7267,7267,7267,7267,7267,7267,7267,7267,7267,7267,7267
unique,7267,3,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4,6531
top,0002-ORFBO,No,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,
freq,1,5174,3675,3749,5086,6560,3495,3198,3608,3182,3195,3582,2896,2870,4005,4311,2445,11


In [154]:
df.describe(include=['integer','float'])

,customer_SeniorCitizen,customer_tenure,account_Charges_Monthly
count,"7,267.00","7,267.00","7,267.00"
mean,0.16,32.35,64.72
std,0.37,24.57,30.13
min,0.00,0.00,18.25
25%,0.00,9.00,35.42
50%,0.00,29.00,70.30
75%,0.00,55.00,89.88
max,1.00,72.00,118.75


Verificando linhas duplicadas

In [155]:
df.duplicated().sum()

0

### Criação de um DataFrame para armazenar informações de verificação

In [156]:
df_check = pd.DataFrame({'coluna': df.columns})

# Verificando quantos valores unicos existem em cada  (7267 é o número máximo possível)
df_check['nunique'] = df.nunique().values

# Verificando quantidades de nulos
df_check['isnull'] = df.isnull().sum().values

# Verificando espações em branco e valores faltantes
temp_series1 = df[df.select_dtypes('object').columns].apply(lambda x: x.str.strip().isin(['']).sum()) # colunas dados object (str)
temp_series2 = df.select_dtypes(['float', 'integer']).isnull().sum() # colunas dados numericos (int e float)
temp_series = pd.concat([temp_series1, temp_series2])
temp_series.name = 'blank'
df_check = df_check.merge(temp_series, how='left', left_on='coluna', right_index=True)

# Verificando data type
df_check['dtypes'] = df.dtypes.values

# Vericando quais os valores unicos para colunas com 5 ou menos valores unicos
df_check['unique'] = df.apply(lambda x: x.unique() if x.nunique() <= 5 else '-').values

# Classificando o tipo das variaveis por coluna
coluna_id = ['customerID']
colunas_quantitativas = ['customer_tenure', 'account_Charges_Monthly', 'account_Charges_Total']
colunas_qualitativas = df.drop(colunas_quantitativas + coluna_id, axis=1).columns.tolist()

dict_class_var = { # dicionario classificacao do tipo de variavel
    'QL': 'variavel qualitativa',
    'QT': 'variavel quantitativa',
    'ID': 'variavel de identificacao'
    }

df_check['class_var'] = df_check['coluna'].replace(colunas_qualitativas, 'QL').replace(
    colunas_quantitativas, 'QT').replace(coluna_id, 'ID' )

# Adicionando coluna de informacoes sobre a coluna
dict_info_cols = { # dicionario informacao das colunas
    'customerID': 'número de identificação único de cada cliente',
    'Churn': 'se o cliente deixou ou não a empresa',
    'gender': 'gênero (masculino e feminino)' ,
    'SeniorCitizen': 'informação sobre um cliente ter ou não idade igual ou maior que 65 anos',
    'Partner':  'se o cliente possui ou não um parceiro ou parceira',
    'Dependents': 'se o cliente possui ou não dependentes',
    'tenure':  'meses de contrato do cliente',
    'PhoneService': 'assinatura de serviço telefônico' ,
    'MultipleLines': 'assisnatura de mais de uma linha de telefone' ,
    'InternetService': 'assinatura de um provedor internet' ,
    'OnlineSecurity': 'assinatura adicional de segurança online' ,
    'OnlineBackup': 'assinatura adicional de backup online' ,
    'DeviceProtection': 'assinatura adicional de proteção no dispositivo' ,
    'TechSupport': 'assinatura adicional de suporte técnico, menos tempo de espera',
    'StreamingTV': 'assinatura de TV a cabo',
    'StreamingMovies': 'assinatura de streaming de filmes' ,
    'Contract': 'tipo de contrato',
    'PaperlessBilling': 'se o cliente prefere receber online a fatura',
    'PaymentMethod': 'forma de pagamento',
    'Charges_Monthly': 'total de todos os serviços do cliente por mês',
    'Charges_Total': 'total gasto pelo cliente'
    }
    
info = df_check['coluna'].copy()
for k in dict_info_cols.keys():
    info.mask(info.str.endswith(k), dict_info_cols[k], inplace=True)
df_check['info'] = info

# Display DF
df_check

,coluna,nunique,isnull,blank,dtypes,unique,class_var,info
0,customerID,7267,0,0,object,-,ID,número de identificação único de cada cliente
1,Churn,3,0,224,object,"[No, Yes, ]",QL,se o cliente deixou ou não a empresa
2,customer_gender,2,0,0,object,"[Female, Male]",QL,gênero (masculino e feminino)
3,customer_SeniorCitizen,2,0,0,int64,"[0, 1]",QL,informação sobre um cliente ter ou não idade igual ou maior que 65 anos
4,customer_Partner,2,0,0,object,"[Yes, No]",QL,se o cliente possui ou não um parceiro ou parceira
5,customer_Dependents,2,0,0,object,"[Yes, No]",QL,se o cliente possui ou não dependentes
6,customer_tenure,73,0,0,int64,-,QT,meses de contrato do cliente
7,phone_PhoneService,2,0,0,object,"[Yes, No]",QL,assinatura de serviço telefônico
8,phone_MultipleLines,3,0,0,object,"[No, Yes, No phone service]",QL,assisnatura de mais de uma linha de telefone
9,internet_InternetService,3,0,0,object,"[DSL, Fiber optic, No]",QL,assinatura de um provedor internet


O que é possível notar de **erro** com a análise superficial:

  

1.  **Inconsistência no tipo do dado**: algumas das colunas que deveriam ser quantitativas estão com ***dtype object***, que nesse caso representa que não são todas as variáveis que estão em modo numérico, seja ***integer*** ou ***float***.
    

2. **Dados em branco**: a coluna **Churn** apresenta um variável única que é um ***string*** em branco (ou vazia) e a coluna **account_Charges_Total**, também apresenta dados em branco, o que pode ser o motivo da inconsistência no tipo do dado, que deveria ser numero (***integer*** ou ***float***).


O que pode ser feito para resolver os problemas apontados:

1.  No caso da coluna **Churn**, apesar de os dados vazios não serem uma amostra tão expressiva (224 linhas de 7267, ou 3%), para não jogar fora essas linhas de informação fora, podemos classificar as colunas vazias como 'sem informação' (em inglês 'no info') e no futuro caso não seja possível preencher essas linhas com dados corretos, será necessário retirá-los do *dataset*.
    

2. Com as coluna **account_Charges_Total**, teremos que investigar melhor o motivo do dados aparecer vazio, e se possível achar o valor correto de forma a calcular usando os dados disponíveis no próprio *dataset*. Deve resolver os dados em branco e o erro no tipo dos dados.

In [157]:
df['Churn'].replace('', 'No Info', inplace=True)
df['Churn'].value_counts()

No         5174
Yes        1869
No Info     224
Name: Churn, dtype: int64

In [158]:
df['account_Charges_Total'] = pd.to_numeric(df['account_Charges_Total'], errors='coerce')
filtro = df['account_Charges_Total'].isnull()
df[colunas_quantitativas][filtro]

,customer_tenure,account_Charges_Monthly,account_Charges_Total
975,0,56.05,NaN
1775,0,20.00,NaN
1955,0,61.90,NaN
2075,0,19.70,NaN
2232,0,20.25,NaN
2308,0,25.35,NaN
2930,0,73.35,NaN
3134,0,25.75,NaN
3203,0,52.55,NaN
4169,0,80.85,NaN


In [159]:
df[colunas_quantitativas].sample(10, random_state=notrandomseed)

,customer_tenure,account_Charges_Monthly,account_Charges_Total
1630,1,19.90,19.90
5777,1,20.55,20.55
3281,12,19.35,212.30
967,50,98.25,"4,858.70"
2505,68,70.80,"4,859.95"
4466,1,19.75,19.75
1309,69,89.95,"6,143.15"
3368,1,52.20,52.20
5944,39,105.65,"4,284.80"
2998,51,19.10,"1,007.80"


In [160]:
df['account_Charges_Total_calc'] = df['account_Charges_Monthly'] * df['customer_tenure']
colunas_quantitativas_2 = colunas_quantitativas + ['account_Charges_Total_calc']
df[colunas_quantitativas_2].sample(10, random_state=notrandomseed)

,customer_tenure,account_Charges_Monthly,account_Charges_Total,account_Charges_Total_calc
1630,1,19.90,19.90,19.90
5777,1,20.55,20.55,20.55
3281,12,19.35,212.30,232.20
967,50,98.25,"4,858.70","4,912.50"
2505,68,70.80,"4,859.95","4,814.40"
4466,1,19.75,19.75,19.75
1309,69,89.95,"6,143.15","6,206.55"
3368,1,52.20,52.20,52.20
5944,39,105.65,"4,284.80","4,120.35"
2998,51,19.10,"1,007.80",974.10


Vendo onde a conta de meses de contrata vezes cobrança mensal bate com o cobrança total

In [161]:
df[df['account_Charges_Total'] == df['account_Charges_Total_calc']]['customer_tenure'].value_counts()

1     634
16      1
Name: customer_tenure, dtype: int64

em 634 casos que o valor de cobrança mensal e total são iguais o cliente tem apenas 1 mes de contrato e em 1 caso que a cobrança mensal e total são iguais o cliente tem 16 meses de contrato. A diferença dos outros clientes, que aparentemente é pequena, pode ser por pequenas variações no valor das faturas mensais.

### Analisando uma amostra

In [162]:
df.sample(15, random_state=notrandomseed)

,customerID,Churn,customer_gender,customer_SeniorCitizen,customer_Partner,customer_Dependents,customer_tenure,phone_PhoneService,phone_MultipleLines,internet_InternetService,internet_OnlineSecurity,internet_OnlineBackup,internet_DeviceProtection,internet_TechSupport,internet_StreamingTV,internet_StreamingMovies,account_Contract,account_PaperlessBilling,account_PaymentMethod,account_Charges_Monthly,account_Charges_Total,account_Charges_Total_calc
1630,2320-JRSDE,Yes,Female,0,Yes,Yes,1,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Electronic check,19.90,19.90,19.90
5777,7893-IXHRQ,Yes,Male,0,No,No,1,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Electronic check,20.55,20.55,20.55
3281,4573-JKNAE,No,Male,0,No,Yes,12,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic),19.35,212.30,232.20
967,1357-BIJKI,No,Male,0,Yes,No,50,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,Yes,One year,Yes,Electronic check,98.25,"4,858.70","4,912.50"
2505,3500-NSDOA,No,Male,0,Yes,Yes,68,Yes,Yes,DSL,No,Yes,No,Yes,No,Yes,Two year,No,Credit card (automatic),70.80,"4,859.95","4,814.40"
4466,6124-ACRHJ,No,Female,0,No,No,1,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,19.75,19.75,19.75
1309,1891-FZYSA,Yes,Male,1,Yes,No,69,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,No,Month-to-month,Yes,Electronic check,89.95,"6,143.15","6,206.55"
3368,4678-DVQEO,Yes,Female,0,No,No,1,Yes,No,DSL,No,No,No,Yes,No,No,Month-to-month,Yes,Electronic check,52.20,52.20,52.20
5944,8111-SLLHI,Yes,Male,1,Yes,No,39,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,105.65,"4,284.80","4,120.35"
2998,4163-HFTUK,No,Male,0,No,No,51,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Electronic check,19.10,"1,007.80",974.10
